CHEF GENIUS


In [37]:
from mesa import Agent,Model

ModuleNotFoundError: No module named 'mesa'

In [ ]:
class Agent_CG(Agent):
    def __init__(self):
        print('¡Hola! Soy ChefGenius, ¿en qué puedo ayudarte el día de hoy?') 

In [ ]:
class Model_CG(Model):
    def __init__(self):
        agent = Agent_CG()

In [ ]:
CG = Model_CG()

¡Hola! Soy ChefGenius, ¿en qué puedo ayudarte el día de hoy?


DESARROLLO ALGORITMO DE BÚSQUEDA

1) Primero, crearemos la clase Nodo para representar cada una de las recetas

In [50]:
import heapq

class Nodo:
    def __init__(self, nombre, ingredientes, metodo, tiempo, restricciones, costo=0):
        self.nombre = nombre
        self.ingredientes = ingredientes 
        self.metodo = metodo  
        self.tiempo = tiempo 
        self.restricciones = restricciones  
        self.costo = costo  
        self.padre = None  
    
    def __lt__(self, otro):
        return self.costo < otro.costo

2) Ahora, se crea la función que determina la heurística de cada Nodo (receta)

In [51]:
def heuristica(nodo, preferencias_usuario):
    penalizacion = 0
    
    
    for restriccion in nodo.restricciones:
        if restriccion in preferencias_usuario['restricciones']:
            penalizacion += 10  

    if nodo.tiempo > preferencias_usuario['max_tiempo']:
        penalizacion += (nodo.tiempo - preferencias_usuario['max_tiempo']) * 2
    
    for ingrediente in nodo.ingredientes:
        if ingrediente not in preferencias_usuario['ingredientes_disponibles']:
            penalizacion += 5 
    
    return penalizacion

3. Se implementa la función la cual devuelve el camino recorrido para llegar hasta la receta final

In [52]:
def reconstruir_camino(nodo):
    camino = []
    while nodo is not None:
        camino.append(nodo.nombre)
        nodo = nodo.padre
    return camino[::-1]

4. Luego, implementamos el algoritmo "A star" o A*, el cual hará una búsqueda por todas las recetas, que tengan una heurística menor, para llegar a la mejor receta posible teniendo en cuenta las preferencias, restricciones y alimentos yu utensilios disponibles por el usuario.


In [53]:
def a_star(inicial, recetas, preferencias_usuario):
    cola_prioridad = []
    lista_recetas = []
    recorridos = {}
    
    heapq.heappush(cola_prioridad, (0, inicial))
    
    visitados = set()
    
    while cola_prioridad:
        costo_actual, nodo_actual = heapq.heappop(cola_prioridad)
        
        if heuristica(nodo_actual, preferencias_usuario) == 0:
            lista_recetas.append(nodo_actual.nombre)
            recorridos[nodo_actual.nombre] = reconstruir_camino(nodo_actual)
        visitados.add(nodo_actual.nombre)
        
        for receta in recetas:
            if receta.nombre not in visitados:
                nuevo_costo = costo_actual + heuristica(receta, preferencias_usuario)
                receta.costo = nuevo_costo
                receta.padre = nodo_actual
                heapq.heappush(cola_prioridad, (nuevo_costo, receta))
    
    if lista_recetas:
        return lista_recetas, recorridos
    else:
        return None  

5) Ahora, pondremos las recetas y las preferencias del usuario para poder ejecutar el algoritmo


In [56]:
recetas = [
    Nodo('Ensalada', ['lechuga', 'tomate', 'aceite'], 'crudo', 20, ['vegano']),
    Nodo('Pizza', ['harina', 'queso', 'tomate'], 'horneado', 20, ['sin gluten']),
    Nodo('Sopa', ['zanahoria', 'papa', 'agua'], 'hervido', 20, ['vegetariano']),
    Nodo('Tacos', ['tortilla', 'carne', 'aguacate'], 'frito', 15, ['sin gluten', 'carnívoro']),
]

preferencias_usuario = {
    'ingredientes_disponibles': ['zanahoria', 'papa', 'agua'],
    'restricciones': ['sin salsa'],
    'max_tiempo': 30
}

6) Por último, ejecutaremos el código 

In [57]:
receta_inicial = recetas[0]  
lista_recetas, lista_caminos_optimos = a_star(receta_inicial, recetas, preferencias_usuario)

if lista_recetas and lista_caminos_optimos:
    print("Recetas disponibles: ")
    for receta in recetas:
        print(receta.nombre)
    print("-----------")
    print("Recetas sugeridas:")
    for r in lista_recetas:
        print("Nombre: ", r)
        print("Camino recorrido: ", lista_caminos_optimos[r])
else:
    print("No se encontraron recetas que cumplan con las preferencias del usuario.")

Recetas disponibles: 
Ensalada
Pizza
Sopa
Tacos
-----------
Recetas sugeridas:
Nombre:  Sopa
Camino recorrido:  ['Ensalada', 'Sopa']
